### Import Required Libraries and Set Up Environment Variables

In [18]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

### CME Data

In [20]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Build URL for CME
query_url_CME = f"{base_url}{CME}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"


In [ ]:
# Debugging step: Confirm the cell is running
print("Starting the API request...")

# Function to generate a list of date ranges
from datetime import timedelta, datetime

def get_date_ranges(start_date, end_date, delta=timedelta(days=30)):
    current_date = start_date
    while current_date < end_date:
        yield current_date, min(current_date + delta, end_date)
        current_date += delta

# Set the start and end dates for the data retrieval
start_date = datetime(2013, 5, 1)
end_date = datetime(2024, 5, 1)

# Empty DataFrame to store all data
all_data = pd.DataFrame()

# Loop through the smaller date ranges (30 days per request)
for start, end in get_date_ranges(start_date, end_date):
    # Format the dates as strings for the API request
    start_str = start.strftime('%Y-%m-%d')
    end_str = end.strftime('%Y-%m-%d')

    print(f"Fetching data from {start_str} to {end_str}")

    # Build the URL with the smaller date range
    query_url_CME = f"{base_url}{CME}?startDate={start_str}&endDate={end_str}&api_key={NASA_API_KEY}"

    # Make the GET request for the CME URL and set a timeout of 120 seconds
    try:
        cme_response = requests.get(query_url_CME, timeout=120)  # 2 minutes timeout

        # Print the status code to confirm if the request was successful or not
        print(f"Status Code: {cme_response.status_code}")

        if cme_response.status_code == 200:
            print(f"Data from {start_str} to {end_str} retrieved successfully!")
            cme_json = cme_response.json()

            # Convert the JSON data to a DataFrame and append it to all_data
            cme_df = pd.DataFrame(cme_json)
            all_data = pd.concat([all_data, cme_df], ignore_index=True)

        else:
            print(f"Error fetching data for {start_str} to {end_str}: {cme_response.status_code}")

    except requests.exceptions.Timeout:
        print(f"Request timed out for {start_str} to {end_str}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

# Once all requests are done, display the total number of rows collected
print(f"Total rows collected: {len(all_data)}")

# Optional: Preview the first few rows of the collected data
all_data.head()

# Optional: Save the collected data to a CSV file for later use
all_data.to_csv('all_cme_data.csv', index=False)

In [24]:
# Print the number of rows
print(f"Total rows of data collected: {len(all_data)}")

Total rows of data collected: 5733


In [26]:
# Preview the first few rows
all_data.head()

,activityID,catalog,startTime,instruments,sourceLocation,activeRegionNum,note,submissionTime,versionId,link,cmeAnalyses,linkedEvents
0,2013-05-01T03:12:00-CME-001,M2M_CATALOG,2013-05-01T03:12Z,"[{'displayName': 'SOHO: LASCO/C2'}, {'displayN...",,NaN,,2013-08-07T16:54Z,1,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,"[{'isMostAccurate': True, 'time21_5': '2013-05...",[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,M2M_CATALOG,2013-05-02T05:24Z,"[{'displayName': 'SOHO: LASCO/C2'}, {'displayN...",,NaN,,2013-08-07T16:56Z,1,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,"[{'isMostAccurate': True, 'time21_5': '2013-05...",None
2,2013-05-02T14:36:00-CME-001,M2M_CATALOG,2013-05-02T14:36Z,"[{'displayName': 'SOHO: LASCO/C2'}, {'displayN...",,NaN,,2013-08-07T16:57Z,1,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,"[{'isMostAccurate': True, 'time21_5': '2013-05...",None
3,2013-05-03T18:00:00-CME-001,M2M_CATALOG,2013-05-03T18:00Z,"[{'displayName': 'SOHO: LASCO/C2'}, {'displayN...",,NaN,,2013-08-07T17:01Z,1,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,"[{'isMostAccurate': True, 'time21_5': '2013-05...",None
4,2013-05-03T22:36:00-CME-001,M2M_CATALOG,2013-05-03T22:36Z,"[{'displayName': 'SOHO: LASCO/C2'}, {'displayN...",,NaN,,2013-08-08T13:40Z,1,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,"[{'isMostAccurate': True, 'time21_5': '2013-05...",[{'activityID': '2013-05-07T04:37:00-IPS-001'}]


In [28]:
# Check for missing data
print(all_data.isnull().sum())

activityID            0
catalog               0
startTime             0
instruments           0
sourceLocation        0
activeRegionNum    4585
note                  0
submissionTime        0
versionId             0
link                  0
cmeAnalyses          76
linkedEvents       4671
dtype: int64


In [30]:
# Print the colum names
print(all_data.columns)

Index(['activityID', 'catalog', 'startTime', 'instruments', 'sourceLocation',
       'activeRegionNum', 'note', 'submissionTime', 'versionId', 'link',
       'cmeAnalyses', 'linkedEvents'],
      dtype='object')


In [36]:
print(json.dumps(cme_json[:1], indent=4))  # Preview the first CME event in JSON format

[
    {
        "activityID": "2024-04-03T08:09:00-CME-001",
        "catalog": "M2M_CATALOG",
        "startTime": "2024-04-03T08:09Z",
        "instruments": [
            {
                "displayName": "STEREO A: SECCHI/COR2"
            }
        ],
        "sourceLocation": "",
        "activeRegionNum": null,
        "note": "CME seen to the SE in STEREO COR2A. The source is a clear far-sided eruption over the SE limb, visible as a wide opening of field lines in SDO AIA 171/193/211/304 starting around 2024-04-03T06:57Z.",
        "submissionTime": "2024-04-03T19:45Z",
        "versionId": 1,
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/29894/-1",
        "cmeAnalyses": [
            {
                "isMostAccurate": true,
                "time21_5": "2024-04-03T13:50Z",
                "latitude": -17.0,
                "longitude": -115.0,
                "halfAngle": 45.0,
                "speed": 592.0,
                "type": "C",
                "f

In [38]:
all_data.head()  # Preview the first few rows of the collected data

,activityID,catalog,startTime,instruments,sourceLocation,activeRegionNum,note,submissionTime,versionId,link,cmeAnalyses,linkedEvents
0,2013-05-01T03:12:00-CME-001,M2M_CATALOG,2013-05-01T03:12Z,"[{'displayName': 'SOHO: LASCO/C2'}, {'displayN...",,NaN,,2013-08-07T16:54Z,1,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,"[{'isMostAccurate': True, 'time21_5': '2013-05...",[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,M2M_CATALOG,2013-05-02T05:24Z,"[{'displayName': 'SOHO: LASCO/C2'}, {'displayN...",,NaN,,2013-08-07T16:56Z,1,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,"[{'isMostAccurate': True, 'time21_5': '2013-05...",None
2,2013-05-02T14:36:00-CME-001,M2M_CATALOG,2013-05-02T14:36Z,"[{'displayName': 'SOHO: LASCO/C2'}, {'displayN...",,NaN,,2013-08-07T16:57Z,1,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,"[{'isMostAccurate': True, 'time21_5': '2013-05...",None
3,2013-05-03T18:00:00-CME-001,M2M_CATALOG,2013-05-03T18:00Z,"[{'displayName': 'SOHO: LASCO/C2'}, {'displayN...",,NaN,,2013-08-07T17:01Z,1,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,"[{'isMostAccurate': True, 'time21_5': '2013-05...",None
4,2013-05-03T22:36:00-CME-001,M2M_CATALOG,2013-05-03T22:36Z,"[{'displayName': 'SOHO: LASCO/C2'}, {'displayN...",,NaN,,2013-08-08T13:40Z,1,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,"[{'isMostAccurate': True, 'time21_5': '2013-05...",[{'activityID': '2013-05-07T04:37:00-IPS-001'}]


In [40]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[:1], indent=4))

[
    {
        "activityID": "2024-04-03T08:09:00-CME-001",
        "catalog": "M2M_CATALOG",
        "startTime": "2024-04-03T08:09Z",
        "instruments": [
            {
                "displayName": "STEREO A: SECCHI/COR2"
            }
        ],
        "sourceLocation": "",
        "activeRegionNum": null,
        "note": "CME seen to the SE in STEREO COR2A. The source is a clear far-sided eruption over the SE limb, visible as a wide opening of field lines in SDO AIA 171/193/211/304 starting around 2024-04-03T06:57Z.",
        "submissionTime": "2024-04-03T19:45Z",
        "versionId": 1,
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/29894/-1",
        "cmeAnalyses": [
            {
                "isMostAccurate": true,
                "time21_5": "2024-04-03T13:50Z",
                "latitude": -17.0,
                "longitude": -115.0,
                "halfAngle": 45.0,
                "speed": 592.0,
                "type": "C",
                "f

In [42]:
# Convert cme_json to a Pandas DataFrame 
cme_df = pd.DataFrame(cme_json)

# Keep only the columns: activityID, startTime, linkedEvents
cme_df = cme_df[['activityID', 'startTime', 'linkedEvents']]

# Preview the first few rows of the DataFrame to ensure it's correct
cme_df.head()

,activityID,startTime,linkedEvents
0,2024-04-03T08:09:00-CME-001,2024-04-03T08:09Z,None
1,2024-04-03T09:38:00-CME-001,2024-04-03T09:38Z,None
2,2024-04-03T15:05:00-CME-001,2024-04-03T15:05Z,None
3,2024-04-03T18:00:00-CME-001,2024-04-03T18:00Z,None
4,2024-04-03T21:12:00-CME-001,2024-04-03T21:12Z,None


In [44]:
# Remove rows with missing 'linkedEvents' since we can't assign these to GSTs
cme_df_cleaned = cme_df.dropna(subset=['linkedEvents'])

# Preview the cleaned DataFrame to ensure rows with missing 'linkedEvents' are removed
cme_df_cleaned.head()

,activityID,startTime,linkedEvents
37,2024-04-11T07:00:00-CME-001,2024-04-11T07:00Z,[{'activityID': '2024-04-15T17:27:00-IPS-001'}]
40,2024-04-11T17:12:00-CME-001,2024-04-11T17:12Z,[{'activityID': '2024-04-11T16:52:00-FLR-001'}]
42,2024-04-12T02:00:00-CME-001,2024-04-12T02:00Z,[{'activityID': '2024-04-16T09:20:00-IPS-001'}]
46,2024-04-12T18:12:00-CME-001,2024-04-12T18:12Z,[{'activityID': '2024-04-12T17:45:00-FLR-001'}]
47,2024-04-13T05:12:00-CME-001,2024-04-13T05:12Z,[{'activityID': '2024-04-13T04:58:00-FLR-001'}]


In [46]:
# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for i in cme_df_cleaned.index:
    # Get the corresponding value from row i in 'activityID' and 'startTime'
    activityID = cme_df_cleaned.loc[i, 'activityID']
    startTime = cme_df_cleaned.loc[i, 'startTime']
    
    # Get the list of dictionaries from row i in 'linkedEvents'
    linkedEvents = cme_df_cleaned.loc[i, 'linkedEvents']
    
    # Iterate over each dictionary in the linkedEvents list
    for event in linkedEvents:
        # Append a new dictionary to the expanded_rows list for each dictionary item
        expanded_rows.append({
            'activityID': activityID,
            'startTime': startTime,
            'linkedEventID': event.get('activityID')  # Adjust to match the structure of linkedEvents
        })

# Create a new DataFrame from the expanded rows
expanded_df = pd.DataFrame(expanded_rows)

# Preview the new expanded DataFrame
expanded_df.head()

,activityID,startTime,linkedEventID
0,2024-04-11T07:00:00-CME-001,2024-04-11T07:00Z,2024-04-15T17:27:00-IPS-001
1,2024-04-11T17:12:00-CME-001,2024-04-11T17:12Z,2024-04-11T16:52:00-FLR-001
2,2024-04-12T02:00:00-CME-001,2024-04-12T02:00Z,2024-04-16T09:20:00-IPS-001
3,2024-04-12T18:12:00-CME-001,2024-04-12T18:12Z,2024-04-12T17:45:00-FLR-001
4,2024-04-13T05:12:00-CME-001,2024-04-13T05:12Z,2024-04-13T04:58:00-FLR-001


In [48]:
# Create the function to extract 'activityID' from a dictionary
def extract_activityID_from_dict(event_dict):
    try:
        # Attempt to get the 'activityID' from the dictionary
        return event_dict.get('activityID', None)
    except (ValueError, TypeError) as e:
        # Log or print the error for debugging purposes
        print(f"Error extracting activityID: {e}")
        return None  # Return None in case of an error

# Test the function using one row from 'linkedEvents' as an example
# Assuming 'cme_df_cleaned' already exists and has a valid 'linkedEvents' column
example_event = cme_df_cleaned['linkedEvents'].iloc[0][0]  # Get the first dictionary in the first row of 'linkedEvents'

# Apply the function to the example
print(extract_activityID_from_dict(example_event))

2024-04-15T17:27:00-IPS-001


In [52]:
# Use .loc to explicitly assign a new column 'GST_ActivityID'
cme_df_cleaned.loc[:, 'GST_ActivityID'] = cme_df_cleaned['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x[0]))

# Preview the DataFrame to ensure the new column has been added correctly
cme_df_cleaned.head()

,activityID,startTime,linkedEvents,GST_ActivityID
37,2024-04-11T07:00:00-CME-001,2024-04-11T07:00Z,[{'activityID': '2024-04-15T17:27:00-IPS-001'}],2024-04-15T17:27:00-IPS-001
40,2024-04-11T17:12:00-CME-001,2024-04-11T17:12Z,[{'activityID': '2024-04-11T16:52:00-FLR-001'}],2024-04-11T16:52:00-FLR-001
42,2024-04-12T02:00:00-CME-001,2024-04-12T02:00Z,[{'activityID': '2024-04-16T09:20:00-IPS-001'}],2024-04-16T09:20:00-IPS-001
46,2024-04-12T18:12:00-CME-001,2024-04-12T18:12Z,[{'activityID': '2024-04-12T17:45:00-FLR-001'}],2024-04-12T17:45:00-FLR-001
47,2024-04-13T05:12:00-CME-001,2024-04-13T05:12Z,[{'activityID': '2024-04-13T04:58:00-FLR-001'}],2024-04-13T04:58:00-FLR-001


In [54]:
# Remove rows with missing 'GST_ActivityID'
cme_df_cleaned = cme_df_cleaned.dropna(subset=['GST_ActivityID'])

# Preview the cleaned DataFrame to ensure rows with missing 'GST_ActivityID' are removed
cme_df_cleaned.head()

,activityID,startTime,linkedEvents,GST_ActivityID
37,2024-04-11T07:00:00-CME-001,2024-04-11T07:00Z,[{'activityID': '2024-04-15T17:27:00-IPS-001'}],2024-04-15T17:27:00-IPS-001
40,2024-04-11T17:12:00-CME-001,2024-04-11T17:12Z,[{'activityID': '2024-04-11T16:52:00-FLR-001'}],2024-04-11T16:52:00-FLR-001
42,2024-04-12T02:00:00-CME-001,2024-04-12T02:00Z,[{'activityID': '2024-04-16T09:20:00-IPS-001'}],2024-04-16T09:20:00-IPS-001
46,2024-04-12T18:12:00-CME-001,2024-04-12T18:12Z,[{'activityID': '2024-04-12T17:45:00-FLR-001'}],2024-04-12T17:45:00-FLR-001
47,2024-04-13T05:12:00-CME-001,2024-04-13T05:12Z,[{'activityID': '2024-04-13T04:58:00-FLR-001'}],2024-04-13T04:58:00-FLR-001


In [56]:
# Print the datatype of each column in the DataFrame
print(cme_df_cleaned.dtypes)

activityID        object
startTime         object
linkedEvents      object
GST_ActivityID    object
dtype: object


In [58]:
# Convert 'GST_ActivityID' to string format
cme_df_cleaned['GST_ActivityID'] = cme_df_cleaned['GST_ActivityID'].astype(str)

# Convert 'startTime' to datetime format
cme_df_cleaned['startTime'] = pd.to_datetime(cme_df_cleaned['startTime'])

# Rename 'startTime' to 'startTime_CME' and 'activityID' to 'cmeID'
cme_df_cleaned = cme_df_cleaned.rename(columns={'startTime': 'startTime_CME', 'activityID': 'cmeID'})

# Drop the 'linkedEvents' column
cme_df_cleaned = cme_df_cleaned.drop(columns=['linkedEvents'])

# Verify that all steps were executed correctly by checking the first few rows and datatypes
print(cme_df_cleaned.dtypes)  # Verify column datatypes
cme_df_cleaned.head()  # Preview the updated DataFrame

cmeID                          object
startTime_CME     datetime64[ns, UTC]
GST_ActivityID                 object
dtype: object


,cmeID,startTime_CME,GST_ActivityID
37,2024-04-11T07:00:00-CME-001,2024-04-11 07:00:00+00:00,2024-04-15T17:27:00-IPS-001
40,2024-04-11T17:12:00-CME-001,2024-04-11 17:12:00+00:00,2024-04-11T16:52:00-FLR-001
42,2024-04-12T02:00:00-CME-001,2024-04-12 02:00:00+00:00,2024-04-16T09:20:00-IPS-001
46,2024-04-12T18:12:00-CME-001,2024-04-12 18:12:00+00:00,2024-04-12T17:45:00-FLR-001
47,2024-04-13T05:12:00-CME-001,2024-04-13 05:12:00+00:00,2024-04-13T04:58:00-FLR-001


In [60]:
# Keep only rows where 'GST_ActivityID' contains 'GST'
cme_gst_df = cme_df_cleaned[cme_df_cleaned['GST_ActivityID'].str.contains('GST', na=False)]

# Preview the filtered DataFrame to ensure the rows contain 'GST'
cme_gst_df.head()

,cmeID,startTime_CME,GST_ActivityID


### GST Data

In [62]:
# Set the base URL to NASA's DONKI API
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST)
GST = "GST"

# Search for GSTs between the start and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
query_url_GST = f"{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"

# Preview the URL to ensure it's correct
print(query_url_GST)


https://api.nasa.gov/DONKI/GST?startDate=2013-05-01&endDate=2024-05-01&api_key=aozPh26Wko4RceUlDcfQvOfLxduydiFVWGvUGAOJ


In [64]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(query_url_GST)

# Check if the request was successful
if gst_response.status_code == 200:
    print("GST data retrieved successfully!")
else:
    print(f"Error: {gst_response.status_code}")

GST data retrieved successfully!


In [66]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()

# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json[:1], indent=4))

[
    {
        "gstID": "2013-06-01T01:00:00-GST-001",
        "startTime": "2013-06-01T01:00Z",
        "allKpIndex": [
            {
                "observedTime": "2013-06-01T01:00Z",
                "kpIndex": 6.0,
                "source": "NOAA"
            }
        ],
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
        "linkedEvents": [
            {
                "activityID": "2013-05-31T15:45:00-HSS-001"
            }
        ],
        "submissionTime": "2013-07-15T19:26Z",
        "versionId": 1
    }
]


In [74]:
# Convert gst_json to a Pandas DataFrame
gst_df = pd.DataFrame(gst_json)

# Keep only the columns: 'gstID', 'startTime', 'linkedEvents'
gst_df = gst_df[['gstID', 'startTime', 'linkedEvents']]

# Preview the first few rows to ensure the DataFrame is correct
gst_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-06-29T03:00:00-GST-001,2013-06-29T03:00Z,None
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...


In [76]:
# Remove rows with missing 'linkedEvents' since we can't assign these to CMEs
gst_df_cleaned = gst_df.dropna(subset=['linkedEvents'])

# Preview the cleaned DataFrame to ensure rows with missing 'linkedEvents' are removed
gst_df_cleaned.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
5,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...


In [78]:
# Use the explode method to ensure that each row is one element in 'linkedEvents'
gst_df_exploded = gst_df_cleaned.explode('linkedEvents').reset_index(drop=True)

# Remove any remaining rows with missing 'linkedEvents' after the explode operation
gst_df_exploded = gst_df_exploded.dropna(subset=['linkedEvents'])

# Preview the DataFrame to ensure the rows have been exploded correctly
gst_df_exploded.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'}
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'}
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'}
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'}
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'}


In [80]:
# Apply the extract_activityID_from_dict function to extract the CME 'activityID' from the 'linkedEvents' column
gst_df_exploded.loc[:, 'CME_ActivityID'] = gst_df_exploded['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))

# Remove rows with missing 'CME_ActivityID' since we can't assign these to CMEs
gst_df_exploded = gst_df_exploded.dropna(subset=['CME_ActivityID'])

# Preview the DataFrame to ensure the new column is added and rows with missing 'CME_ActivityID' are removed
gst_df_exploded.head()

,gstID,startTime,linkedEvents,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'},2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'},2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'},2013-10-02T02:47:00-MPC-001


In [82]:
# Convert 'CME_ActivityID' to string format
gst_df_exploded['CME_ActivityID'] = gst_df_exploded['CME_ActivityID'].astype(str)

# Convert 'gstID' to string format
gst_df_exploded['gstID'] = gst_df_exploded['gstID'].astype(str)

# Convert 'startTime' to datetime format
gst_df_exploded['startTime'] = pd.to_datetime(gst_df_exploded['startTime'])

# Rename 'startTime' to 'startTime_GST'
gst_df_exploded = gst_df_exploded.rename(columns={'startTime': 'startTime_GST'})

# Drop the 'linkedEvents' column as it's no longer needed
gst_df_exploded = gst_df_exploded.drop(columns=['linkedEvents'])

# Verify that all steps were executed correctly
print(gst_df_exploded.dtypes)  # Check the data types of all columns
gst_df_exploded.head()  # Preview the first few rows of the DataFrame

gstID                          object
startTime_GST     datetime64[ns, UTC]
CME_ActivityID                 object
dtype: object


,gstID,startTime_GST,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01 01:00:00+00:00,2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-10-02T02:47:00-MPC-001


In [84]:
# Keep only rows where 'CME_ActivityID' contains 'CME'
gst_df_filtered = gst_df_exploded[gst_df_exploded['CME_ActivityID'].str.contains('CME', na=False)]

# Preview the filtered DataFrame to ensure the rows contain 'CME'
gst_df_filtered.head()

,gstID,startTime_GST,CME_ActivityID
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
5,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
7,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
9,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001


### Merge both datatsets

In [113]:
# Merge GST and CME datasets after ensuring data exists
merged_df = pd.merge(
    gst_df_filtered,
    cme_df_cleaned, 
    left_on=['gstID', 'CME_ActivityID'],
    right_on=['GST_ActivityID', 'cmeID'],
    how='inner'
)

# Check number of rows in merged DataFrame
print(f"Number of rows in Merged DataFrame: {len(merged_df)}")

Number of rows in Merged DataFrame: 0


In [115]:
# Check unique values in key columns for both datasets
print("Unique gstID and CME_ActivityID in GST data:")
print(gst_df_filtered[['gstID', 'CME_ActivityID']].drop_duplicates())

print("\nUnique GST_ActivityID and cmeID in CME data:")
print(cme_df_cleaned[['GST_ActivityID', 'cmeID']].drop_duplicates())


Unique gstID and CME_ActivityID in GST data:
                           gstID               CME_ActivityID
1    2013-06-07t03:00:00-gst-001  2013-06-02t20:24:00-cme-001
2    2013-10-02t03:00:00-gst-001  2013-09-29t22:40:00-cme-001
5    2013-12-08t00:00:00-gst-001  2013-12-04t23:12:00-cme-001
7    2014-02-19t03:00:00-gst-001  2014-02-16t14:15:00-cme-001
9    2014-02-20t03:00:00-gst-001  2014-02-18t01:25:00-cme-001
..                           ...                          ...
192  2023-12-18t06:00:00-gst-001  2023-12-14t17:38:00-cme-001
195  2024-03-03t18:00:00-gst-001  2024-02-28t17:48:00-cme-001
199  2024-03-24t12:00:00-gst-001  2024-03-23t01:25:00-cme-001
200  2024-03-24t12:00:00-gst-001  2024-03-23t01:48:00-cme-001
203  2024-04-19t18:00:00-gst-001  2024-04-15t06:48:00-cme-001

[61 rows x 2 columns]

Unique GST_ActivityID and cmeID in CME data:
Empty DataFrame
Columns: [GST_ActivityID, cmeID]
Index: []


In [123]:
# Re-fetch the CME data and check if it's returning the expected data
startDate = "2023-01-01"
endDate = "2023-01-07"  # Try a smaller range to see if the request works
query_url_CME = f"{base_url}{CME}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"
cme_response = requests.get(query_url_CME, timeout=60)

# Convert response to DataFrame
cme_json = cme_response.json()
cme_df = pd.DataFrame(cme_json)

# Check the first few rows to verify the data
print(cme_df.head())


                    activityID      catalog          startTime  \
0  2023-01-01T02:48:00-CME-001  M2M_CATALOG  2023-01-01T02:48Z   
1  2023-01-01T20:24:00-CME-001  M2M_CATALOG  2023-01-01T20:24Z   
2  2023-01-02T06:00:00-CME-001  M2M_CATALOG  2023-01-02T06:00Z   
3  2023-01-02T08:48:00-CME-001  M2M_CATALOG  2023-01-02T08:48Z   
4  2023-01-02T11:00:00-CME-001  M2M_CATALOG  2023-01-02T11:00Z   

                                         instruments sourceLocation  \
0  [{'displayName': 'SOHO: LASCO/C2'}, {'displayN...                  
1  [{'displayName': 'SOHO: LASCO/C2'}, {'displayN...                  
2  [{'displayName': 'SOHO: LASCO/C2'}, {'displayN...                  
3  [{'displayName': 'SOHO: LASCO/C2'}, {'displayN...                  
4  [{'displayName': 'SOHO: LASCO/C2'}, {'displayN...                  

   activeRegionNum                                               note  \
0              NaN  Visible in east of SOHO LASCO C2/C3 and STEREO...   
1              NaN  Partial ha

In [ ]:
# Merge the GST and CME datasets
merged_df = pd.merge(
    gst_df_filtered,  # GST dataset
    cme_df_cleaned,   # CME dataset
    left_on=['gstID', 'CME_ActivityID'],  # Columns in GST data
    right_on=['GST_ActivityID', 'cmeID'],  # Columns in CME data
    how='inner'  # Use 'inner' to keep only matching rows, or 'outer' for all rows
)

# Preview the merged DataFrame to ensure the merge worked as expected
merged_df.head()

In [102]:
# Extract just the date part of the CME_ActivityID and GST_ActivityID for comparison
gst_df_filtered['CME_Date'] = gst_df_filtered['CME_ActivityID'].str[:10]
cme_df_cleaned['CME_Date'] = cme_df_cleaned['cmeID'].str[:10]

# Preview the new columns
print(gst_df_filtered[['CME_ActivityID', 'CME_Date']].head())
print(cme_df_cleaned[['cmeID', 'CME_Date']].head())


                CME_ActivityID    CME_Date
1  2013-06-02t20:24:00-cme-001  2013-06-02
2  2013-09-29t22:40:00-cme-001  2013-09-29
5  2013-12-04t23:12:00-cme-001  2013-12-04
7  2014-02-16t14:15:00-cme-001  2014-02-16
9  2014-02-18t01:25:00-cme-001  2014-02-18
Empty DataFrame
Columns: [cmeID, CME_Date]
Index: []


In [110]:
# Re-check the CME data retrieval
query_url_CME = f"{base_url}{CME}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"
cme_response = requests.get(query_url_CME)
cme_json = cme_response.json()

# Convert to DataFrame
cme_df = pd.DataFrame(cme_json)

# Preview the first few rows of the new CME data
cme_df.head()


KeyboardInterrupt: 

In [98]:
# Check the number of rows in each DataFrame
print(f"Number of rows in CME DataFrame: {len(cme_df_cleaned)}")
print(f"Number of rows in GST DataFrame: {len(gst_df_filtered)}")
print(f"Number of rows in Merged DataFrame: {len(merged_df)}")

# Verify if the number of rows in the merged DataFrame matches those in cme and gst DataFrames
if len(merged_df) == len(cme_df_cleaned) and len(merged_df) == len(gst_df_filtered):
    print("The merged DataFrame has the same number of rows as the original CME and GST DataFrames.")
else:
    print("The merged DataFrame does not have the same number of rows as the original CME and GST DataFrames.")

Number of rows in CME DataFrame: 30
Number of rows in GST DataFrame: 61
Number of rows in Merged DataFrame: 0
The merged DataFrame does not have the same number of rows as the original CME and GST DataFrames.


### Computing the time it takes for a CME to cause a GST

In [31]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.


In [32]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 


### Exporting data in csv format

In [34]:
# Export data to CSV without the index
